# SC4001 Deep Learning - Group Project
## Oxford Flowers 102 Recognition

### Role: Person C - Advanced Techniques & Analysis

This notebook contains:
1. Few-Shot Learning experiments (1, 5, 10 samples per class)
2. Advanced Techniques (MixUp, Triplet Loss)
3. Performance comparison and reporting

## Part 1: Imports and Setup

In [10]:
import os
import json
import random
import copy
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Subset, Dataset

from torchvision import datasets, transforms
from tqdm.notebook import tqdm
from baseline_model import (
	set_seed,
	get_flowers_dataloaders,
	train_epoch,
	validate,
	create_model_with_timeout
)


Dataset found at: ../data/flowers-102
Loading Flowers102 dataset...


RuntimeError: Dataset not found or corrupted. You can use download=True to download it

In [ ]:
try:
    from baseline_model import (
        set_seed,
        get_data_transforms,
        create_baseline_model,
        train_epoch,
        validate,
        create_model_with_timeout,
        get_flowers_dataloaders 
    )
    print("Successfully imported from baseline_model.py")
except ImportError:
    print("ERROR: Could not import from baseline_model.py")
    print("Please ensure 'baseline_model.py' is in the same folder as this notebook.")

print("\nPerson C - Advanced Techniques Module Loaded")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
set_seed(42)
print(f"Using device: {device}")


DATA_PATH = '../data'

ERROR: Could not import from baseline_model.py
Please ensure 'baseline_model.py' is in the same folder as this notebook.

Person C - Advanced Techniques Module Loaded


NameError: name 'set_seed' is not defined

## Part 2: Few-Shot Learning

Code for creating k-shot datasets and training the models.

In [ ]:
class FewShotDataset:
    """Create few-shot subsets of the training data"""
    
    def __init__(self, full_dataset, k_shot, num_classes=102, seed=42):
        """
        Args:
            full_dataset: Original training dataset (MUST be loaded with transform=None)
            k_shot: Number of samples per class (1, 5, or 10)
            num_classes: Total number of classes
            seed: Random seed for reproducibility
        """
        self.full_dataset = full_dataset
        self.k_shot = k_shot
        self.num_classes = num_classes
        
        random.seed(seed)
        np.random.seed(seed)
        
        # Group indices by class
        class_indices = defaultdict(list)
        
        # Use _labels attribute from Flowers102 dataset for efficiency
        try:
            labels = self.full_dataset._labels
            for idx, label in enumerate(labels):
                class_indices[label].append(idx)
        except AttributeError:
            print("Warning: Could not use ._labels, iterating dataset slowly...")
            for idx in range(len(full_dataset)):
                _, label = full_dataset[idx] 
                class_indices[label].append(idx)
        
        # Sample k images per class
        self.selected_indices = []
        for class_id in range(num_classes):
            if len(class_indices[class_id]) >= k_shot:
                sampled = random.sample(class_indices[class_id], k_shot)
                self.selected_indices.extend(sampled)
            else:
                # Safeguard if a class has < k_shot samples (shouldn't happen for 1, 5, 10)
                print(f"Warning: Class {class_id} has fewer than {k_shot} samples. Using all.")
                self.selected_indices.extend(class_indices[class_id])
        
        print(f"\nFew-Shot Dataset Created:")
        print(f"  K-shot: {k_shot}")
        print(f"  Total samples: {len(self.selected_indices)}")
        print(f"  Expected: {num_classes * k_shot}")
    
    def get_subset(self, target_dataset):
        """Return PyTorch Subset *from the target_dataset* using selected indices"""
        return Subset(target_dataset, self.selected_indices)

In [ ]:
def create_fewshot_dataloaders(k_shot, batch_size=32, num_workers=4):
    """
    Create dataloaders for few-shot learning
    """
    train_transform, test_transform = get_data_transforms()
    
    # Load a RAW version of the training set (transform=None) 
    # This is ONLY for sampling indices based on labels.
    train_dataset_raw = datasets.Flowers102(
        root=DATA_PATH,
        split='train',
        download=True,
        transform=None
    )
    
    # Load the TRANSFORMED version of the training set.
    # This is the dataset we will actually train on.
    train_dataset_full = datasets.Flowers102(
        root=DATA_PATH,
        split='train',
        download=False, # Already downloaded
        transform=train_transform
    )
    
    # Create few-shot sampler using the RAW dataset to get indices
    fewshot_sampler = FewShotDataset(train_dataset_raw, k_shot=k_shot)
    # Create the final Subset using the TRANSFORMED dataset
    train_subset = fewshot_sampler.get_subset(train_dataset_full)
    
    # Validation and test remain the same
    val_dataset = datasets.Flowers102(
        root=DATA_PATH,
        split='val',
        download=True,
        transform=test_transform
    )
    
    test_dataset = datasets.Flowers102(
        root=DATA_PATH,
        split='test',
        download=True,
        transform=test_transform
    )
    
    pin_mem = True if torch.cuda.is_available() else False
    num_workers = 2 if os.name == 'nt' else num_workers
    
    # Create dataloaders
    train_loader = DataLoader(
        train_subset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=pin_mem
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=pin_mem
    )
    
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=pin_mem
    )
    
    return train_loader, val_loader, test_loader

In [ ]:
def train_fewshot_model(k_shot, model_name='resnet18', num_epochs=50, lr=0.001):
    """
    Train model with few-shot learning
    """
    print(f"\n{'='*60}")
    print(f"Training {k_shot}-Shot Learning Model (Arch: {model_name})")
    print(f"{'='*60}")
    
    # Create few-shot dataloaders
    train_loader, val_loader, test_loader = create_fewshot_dataloaders(
        k_shot=k_shot,
        batch_size=32
    )
    
    # Create model
    if 'vit' in model_name:
        model = create_model_with_timeout(model_name, num_classes=102, device=device)
    else:
        model = create_baseline_model(num_classes=102, pretrained=True, model_name=model_name)
        model = model.to(device)
    
    # Training setup
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    
    # Training loop
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }
    
    best_val_acc = 0.0
    best_model_state = None
    
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        
        # Train
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        
        # Validate
        val_loss, val_acc = validate(model, val_loader, criterion, device)
        
        # Scheduler step
        scheduler.step()
        
        # Save history
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = copy.deepcopy(model.state_dict())
            print(f"  ✓ New best! Val Acc: {val_acc:.2f}%")
        
        print(f"  Train: Loss={train_loss:.4f}, Acc={train_acc:.2f}%")
        print(f"  Val: Loss={val_loss:.4f}, Acc={val_acc:.2f}%")
    
    # Load best model and evaluate on test
    model.load_state_dict(best_model_state)
    test_loss, test_acc = validate(model, test_loader, criterion, device, valid_or_test="test")
    
    history['best_val_acc'] = best_val_acc
    history['test_loss'] = test_loss
    history['test_acc'] = test_acc
    history['k_shot'] = k_shot
    
    print(f"\n{'='*60}")
    print(f"{k_shot}-Shot Results ({model_name}):")
    print(f"  Best Val Acc: {best_val_acc:.2f}%")
    print(f"  Test Acc: {test_acc:.2f}%")
    print(f"{'='*60}")
    
    return model, history

## Part 3: MixUp Augmentation

In [ ]:
def mixup_data(x, y, alpha=1.0, device='cuda'):
    """
    Apply MixUp augmentation
    """
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(device)
    
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    
    return mixed_x, y_a, y_b, lam


def mixup_criterion(criterion, pred, y_a, y_b, lam):
    """MixUp loss function"""
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)


def train_epoch_mixup(model, dataloader, criterion, optimizer, device, alpha=1.0):
    """Training epoch with MixUp augmentation"""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(dataloader, desc='Training (MixUp)')
    
    for batch_idx, (images, labels) in enumerate(pbar):
        images, labels = images.to(device), labels.to(device)
        
        # Apply MixUp
        mixed_images, labels_a, labels_b, lam = mixup_data(images, labels, alpha, device)
        
        # Forward pass
        optimizer.zero_grad()
        outputs = model(mixed_images)
        loss = mixup_criterion(criterion, outputs, labels_a, labels_b, lam)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Statistics
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += (lam * predicted.eq(labels_a).sum().item() + 
                    (1 - lam) * predicted.eq(labels_b).sum().item())
        
        pbar.set_postfix({
            'loss': running_loss/(batch_idx+1),
            'acc': 100.*correct/total
        })
    
    return running_loss/len(dataloader), 100.*correct/total

In [ ]:
def train_with_mixup(model_name='resnet18', num_epochs=30, alpha=1.0, lr=0.001):
    """
    Train model with MixUp augmentation
    """
    print(f"\n{'='*60}")
    print(f"Training with MixUp (alpha={alpha}, Arch: {model_name})")
    print(f"{'='*60}")
    
    # Import baseline dataloader (this uses the 10-shot train set)
    train_loader, val_loader, test_loader = get_flowers_dataloaders(
        batch_size=32, data_path=DATA_PATH
    )
    
    # Create model
    if 'vit' in model_name:
        model = create_model_with_timeout(model_name, num_classes=102, device=device)
    else:
        model = create_baseline_model(num_classes=102, pretrained=True, model_name=model_name)
        model = model.to(device)
    
    # Training setup
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    
    # Training loop
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }
    
    best_val_acc = 0.0
    best_model_state = None
    
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        
        # Train with MixUp
        train_loss, train_acc = train_epoch_mixup(
            model, train_loader, criterion, optimizer, device, alpha=alpha
        )
        
        # Validate (no MixUp)
        val_loss, val_acc = validate(model, val_loader, criterion, device)
        
        # Scheduler step
        scheduler.step()
        
        # Save history
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = copy.deepcopy(model.state_dict())
            print(f"  ✓ New best! Val Acc: {val_acc:.2f}%")
        
        print(f"  Train: Loss={train_loss:.4f}, Acc={train_acc:.2f}%")
        print(f"  Val: Loss={val_loss:.4f}, Acc={val_acc:.2f}%")
    
    # Evaluate on test set
    model.load_state_dict(best_model_state)
    test_loss, test_acc = validate(model, test_loader, criterion, device, valid_or_test="test")
    
    history['best_val_acc'] = best_val_acc
    history['test_loss'] = test_loss
    history['test_acc'] = test_acc
    history['alpha'] = alpha
    
    print(f"\n{'='*60}")
    print(f"MixUp Results (alpha={alpha}, {model_name}):")
    print(f"  Best Val Acc: {best_val_acc:.2f}%")
    print(f"  Test Acc: {test_acc:.2f}%")
    print(f"{'='*60}")
    
    return model, history

## Part 4: Triplet Loss

In [ ]:
class TripletLoss(nn.Module):
    """Triplet Loss for metric learning"""
    
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin
    
    def forward(self, anchor, positive, negative):
        distance_positive = F.pairwise_distance(anchor, positive, 2)
        distance_negative = F.pairwise_distance(anchor, negative, 2)
        losses = F.relu(distance_positive - distance_negative + self.margin)
        return losses.mean()


class TripletDataset(Dataset):
    """Dataset that returns triplets (anchor, positive, negative)"""
    
    def __init__(self, base_dataset):
        self.base_dataset = base_dataset
        
        # Group indices by class
        self.class_indices = defaultdict(list)
        try:
            labels = self.base_dataset._labels
            for idx, label in enumerate(labels):
                self.class_indices[label].append(idx)
        except AttributeError:
            print("Warning: Iterating slowly for TripletDataset...")
            for idx in range(len(base_dataset)):
                _, label = base_dataset[idx]
                self.class_indices[label].append(idx)
                self.classes = list(self.class_indices.keys())
                self.class_count = len(self.classes)
    def __len__(self):
        return len(self.base_dataset)
    
    def __getitem__(self, idx):
        # Get anchor
        anchor_img, anchor_label = self.base_dataset[idx]
        
        # Get positive (same class)
        positive_indices = self.class_indices[anchor_label]
        positive_idx = idx
        if len(positive_indices) > 1:
            while positive_idx == idx:
                positive_idx = random.choice(positive_indices)
        else:
            positive_idx = idx
            
        positive_img, _ = self.base_dataset[positive_idx]
        
        # Get negative (different class)
        negative_label = anchor_label
        while negative_label == anchor_label:
            negative_label = random.choice(self.classes)
            
        negative_idx = random.choice(self.class_indices[negative_label])
        negative_img, _ = self.base_dataset[negative_idx]
        
        return anchor_img, positive_img, negative_img, anchor_label                

In [ ]:
class TripletModelWrapper(nn.Module):
    """Wraps a base model for triplet loss AND classification"""
    
    def __init__(self, base_model, embedding_dim=128, num_classes=102):
        super(TripletModelWrapper, self).__init__()
        
        # Get feature dimension from the base_model's fc layer
        try:
            num_features = base_model.fc.in_features
            # Overwrite the fc layer to be an embedding layer
            base_model.fc = nn.Linear(num_features, embedding_dim)
        except AttributeError:
            # Handle models like ViT which use 'head'
            num_features = base_model.head.in_features
            base_model.head = nn.Linear(num_features, embedding_dim)
            
        self.base_model = base_model
        
        # Add a *new* classification head after the embedding layer
        self.classifier = nn.Linear(embedding_dim, num_classes)
    
    def forward(self, x, return_embedding=False):
        # Get the embedding from the modified base model
        embedding = self.base_model(x)
        
        if return_embedding:
            return embedding
        
        # Pass the embedding to the new classifier
        logits = self.classifier(embedding)
        return logits, embedding

In [ ]:
def train_epoch_triplet(model, dataloader, criterion_triplet, criterion_cls, 
                        optimizer, device, lambda_triplet=0.5):
    """Training epoch with combined triplet and classification loss"""
    model.train()
    running_loss = 0.0
    running_triplet_loss = 0.0
    running_cls_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(dataloader, desc='Training (Triplet)')
    
    for batch_idx, (anchor, positive, negative, labels) in enumerate(pbar):
        anchor = anchor.to(device)
        positive = positive.to(device)
        negative = negative.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        # Get outputs for all three images
        anchor_out, anchor_emb = model(anchor)
        positive_out, positive_emb = model(positive)
        negative_out, negative_emb = model(negative)
        
        # 1. Compute Triplet loss
        triplet_loss = criterion_triplet(anchor_emb, positive_emb, negative_emb)
        
        # 2. Compute Classification loss (on the anchor)
        cls_loss = criterion_cls(anchor_out, labels)
        
        # 3. Combined loss
        loss = lambda_triplet * triplet_loss + (1 - lambda_triplet) * cls_loss
        
        # Backward
        loss.backward()
        optimizer.step()
        
        # Statistics
        running_loss += loss.item()
        running_triplet_loss += triplet_loss.item()
        running_cls_loss += cls_loss.item()
        
        _, predicted = anchor_out.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        pbar.set_postfix({
            'loss': running_loss/(batch_idx+1),
            'triplet': running_triplet_loss/(batch_idx+1),
            'cls': running_cls_loss/(batch_idx+1),
            'acc': 100.*correct/total
        })
    
    return running_loss/len(dataloader), 100.*correct/total

In [ ]:
def train_with_triplet_loss(model_name='resnet18', num_epochs=30, margin=1.0, 
                            lambda_triplet=0.5, lr=0.001):
    """
    Train model with triplet loss
    """
    print(f"\n{'='*60}")
    print(f"Training with Triplet Loss (margin={margin}, lambda={lambda_triplet}, Arch={model_name})")
    print(f"{'='*60}")
    
    # Load datasets
    train_transform, test_transform = get_data_transforms()
    
    # Note: TripletDataset needs the base dataset *with transforms*
    train_dataset = datasets.Flowers102(
        root=DATA_PATH,
        split='train',
        download=True,
        transform=train_transform
    )
    
    val_dataset = datasets.Flowers102(
        root=DATA_PATH,
        split='val',
        download=True,
        transform=test_transform
    )
    
    test_dataset = datasets.Flowers102(
        root=DATA_PATH,
        split='test',
        download=True,
        transform=test_transform
    )
    
    # Create triplet dataset for training
    triplet_train_dataset = TripletDataset(train_dataset)
    
    num_workers = 2 if os.name == 'nt' else 4
    pin_mem = True if torch.cuda.is_available() else False
    
    train_loader = DataLoader(triplet_train_dataset, batch_size=32, shuffle=True, num_workers=num_workers, pin_memory=pin_mem)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=num_workers, pin_memory=pin_mem)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=num_workers, pin_memory=pin_mem)
    
    # Create base model (e.g., ResNet18)
    if 'vit' in model_name:
        base_model = create_model_with_timeout(model_name, num_classes=102, device=device, pretrained=True)
    else:
        base_model = create_baseline_model(num_classes=102, pretrained=True, model_name=model_name)
    
    # Wrap in triplet model
    model = TripletModelWrapper(base_model, embedding_dim=128, num_classes=102)
    model = model.to(device)
    
    # Training setup
    criterion_triplet = TripletLoss(margin=margin)
    criterion_cls = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    
    # Training loop
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }
    
    best_val_acc = 0.0
    best_model_state = None
    
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        
        # Train with triplet loss
        train_loss, train_acc = train_epoch_triplet(
            model, train_loader, criterion_triplet, criterion_cls,
            optimizer, device, lambda_triplet=lambda_triplet
        )
        
        # Validate (classification only)
        val_loss, val_acc = validate(model, val_loader, criterion_cls, device)
        
        # Scheduler step
        scheduler.step()
        
        # Save history
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = copy.deepcopy(model.state_dict())
            print(f"  ✓ New best! Val Acc: {val_acc:.2f}%")
        
        print(f"  Train: Loss={train_loss:.4f}, Acc={train_acc:.2f}%")
        print(f"  Val: Loss={val_loss:.4f}, Acc={val_acc:.2f}%")
    
    # Evaluate on test set
    model.load_state_dict(best_model_state)
    test_loss, test_acc = validate(model, test_loader, criterion_cls, device, valid_or_test="test")
    
    history['best_val_acc'] = best_val_acc
    history['test_loss'] = test_loss
    history['test_acc'] = test_acc
    history['margin'] = margin
    history['lambda_triplet'] = lambda_triplet
    
    print(f"\n{'='*60}")
    print(f"Triplet Loss Results ({model_name}):")
    print(f"  Best Val Acc: {best_val_acc:.2f}%")
    print(f"  Test Acc: {test_acc:.2f}%")
    print(f"{'='*60}")
    
    return model, history

## Part 5: Experiment Runner

This master function runs all experiments for Person C.

In [ ]:
def run_all_experiments(quick_test=False):
    """
    Run all Person C experiments
    
    Args:
        quick_test: If True, run with reduced epochs for testing
    
    Returns:
        results: Dictionary containing all experiment results
    """
    results = {}
    
    # Set epochs based on mode
    epochs_fewshot = 50 if not quick_test else 3 # 50 epochs for few-shot
    epochs_standard = 30 if not quick_test else 3 # 30 epochs for others
    
    print("\n" + "="*80)
    print("PERSON C - ADVANCED TECHNIQUES EXPERIMENTS")
    print(f"QUICK TEST MODE: {quick_test}")
    print("="*80)
    
    # Define base model for comparison
    # We use resnet18 to compare against Person A's baseline
    # We can also run this with 'vit_base_patch32_224' to compare against Person B
    BASE_MODEL_NAME = 'resnet18'
    
    #EXPERIMENT 1: Few-Shot Learning
    print("\n\n EXPERIMENT 1: Few-Shot Learning \n")
    
    fewshot_results = {}
    for k_shot in [1, 5, 10]:
        try:
            model, history = train_fewshot_model(
                k_shot=k_shot,
                model_name=BASE_MODEL_NAME,
                num_epochs=epochs_fewshot,
                lr=0.001
            )
            fewshot_results[f'{k_shot}_shot'] = history
            
            # Save model
            save_dir = f'results/person_c/fewshot_{k_shot}shot_{BASE_MODEL_NAME}'
            os.makedirs(save_dir, exist_ok=True)
            torch.save(model.state_dict(), f'{save_dir}/model.pth')
            
        except Exception as e:
            print(f"ERROR in {k_shot}-shot learning: {e}")
            if "baseline_model" in str(e):
                print("*** HINT: 'baseline_model.py' not found. Stopping. ***")
                break 
    
    results['fewshot'] = fewshot_results
    
    #EXPERIMENT 2: MixUp Augmentation
    print("\n\n EXPERIMENT 2: MixUp Augmentation \n")
    
    mixup_results = {}
    # Run only on alpha=1.0 for the report, as specified in handout
    # You can add [0.2, 0.5, 1.0] to test more
    for alpha in [1.0]: 
        try:
            model, history = train_with_mixup(
                model_name=BASE_MODEL_NAME,
                num_epochs=epochs_standard,
                alpha=alpha,
                lr=0.001
            )
            mixup_results[f'alpha_{alpha}'] = history
            
            # Save model
            save_dir = f'results/person_c/mixup_alpha{alpha}_{BASE_MODEL_NAME}'
            os.makedirs(save_dir, exist_ok=True)
            torch.save(model.state_dict(), f'{save_dir}/model.pth')
            
        except Exception as e:
            print(f"ERROR in MixUp (alpha={alpha}): {e}")
            if "baseline_model" in str(e):
                print("*** HINT: 'baseline_model.py' not found. Stopping. ***")
                break
    
    results['mixup'] = mixup_results
    
    #EXPERIMENT 3: Triplet Loss
    print("\n\n EXPERIMENT 3: Triplet Loss \n")
    
    try:
        model, history = train_with_triplet_loss(
            model_name=BASE_MODEL_NAME,
            num_epochs=epochs_standard,
            margin=1.0,
            lambda_triplet=0.5,
            lr=0.001
        )
        results['triplet'] = history
        
        # Save model
        save_dir = f'results/person_c/triplet_{BASE_MODEL_NAME}'
        os.makedirs(save_dir, exist_ok=True)
        torch.save(model.state_dict(), f'{save_dir}/model.pth')
        
    except Exception as e:
        print(f"ERROR in Triplet Loss: {e}")
        if "baseline_model" in str(e):
            print("*** HINT: 'baseline_model.py' not found. Stopping. ***")
            
    # Save all results
    os.makedirs('results/person_c', exist_ok=True)
    with open(f'results/person_c/all_results_{BASE_MODEL_NAME}.json', 'w') as f:
        json.dump(results, f, indent=4)
    
    print("\n" + "="*80)
    print("ALL EXPERIMENTS COMPLETED!")
    print("="*80)
    
    return results

## Part 6: Visualization and Reporting

In [ ]:
def plot_fewshot_comparison(fewshot_results, baseline_acc=None):
    """Plot few-shot learning comparison"""
    if not fewshot_results:
        print("No few-shot results to plot.")
        return
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Validation accuracy
    for k_name, history in fewshot_results.items():
        val_acc = history['val_acc']
        axes[0].plot(val_acc, label=k_name, linewidth=2)
    
    axes[0].set_xlabel('Epoch', fontsize=12)
    axes[0].set_ylabel('Validation Accuracy (%)', fontsize=12)
    axes[0].set_title('Few-Shot Learning: Validation Accuracy', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Test accuracy bar chart
    k_shots = []
    test_accs = []
    for k_name, history in fewshot_results.items():
        k_shots.append(k_name)
        test_accs.append(history['test_acc'])
    
    axes[1].bar(k_shots, test_accs, color=['#1f77b4', '#ff7f0e', '#2ca02c'])
    if baseline_acc:
        axes[1].axhline(y=baseline_acc, color='r', linestyle='--', label=f'10-shot Baseline ({baseline_acc:.2f}%)')
        axes[1].legend()
        
    axes[1].set_xlabel('K-Shot', fontsize=12)
    axes[1].set_ylabel('Test Accuracy (%)', fontsize=12)
    axes[1].set_title('Few-Shot Learning: Test Accuracy', fontsize=14, fontweight='bold')
    axes[1].grid(True, alpha=0.3, axis='y')
    
    # Add values on bars
    for i, v in enumerate(test_accs):
        axes[1].text(i, v + 1, f'{v:.2f}%', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('results/person_c/fewshot_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
def plot_mixup_comparison(mixup_results, baseline_acc=None):
    """Plot MixUp augmentation comparison"""
    if not mixup_results:
        print("No MixUp results to plot.")
        return
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Validation accuracy
    for alpha_name, history in mixup_results.items():
        val_acc = history['val_acc']
        axes[0].plot(val_acc, label=alpha_name, linewidth=2)
    

    axes[0].set_xlabel('Epoch', fontsize=12)
    axes[0].set_ylabel('Validation Accuracy (%)', fontsize=12)
    axes[0].set_title('MixUp Augmentation: Validation Accuracy', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Test accuracy bar chart
    alphas = []
    test_accs = []
    for alpha_name, history in mixup_results.items():
        alphas.append(alpha_name.replace('alpha_', 'α='))
        test_accs.append(history['test_acc'])
    
    axes[1].bar(alphas, test_accs, color=['#9467bd', '#8c564b', '#e377c2'])
    if baseline_acc:
        axes[1].axhline(y=baseline_acc, color='r', linestyle='--', label=f'Baseline ({baseline_acc:.2f}%)')
        axes[1].legend()
        
    axes[1].set_xlabel('MixUp Alpha', fontsize=12)
    axes[1].set_ylabel('Test Accuracy (%)', fontsize=12)
    axes[1].set_title('MixUp Augmentation: Test Accuracy', fontsize=14, fontweight='bold')
    axes[1].grid(True, alpha=0.3, axis='y')
    
    # Add values on bars
    for i, v in enumerate(test_accs):
        axes[1].text(i, v + 1, f'{v:.2f}%', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('results/person_c/mixup_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
def create_comparison_table(results, baseline_acc=79.67, person_b_best_acc=None):
    """
    Create comprehensive comparison table
    """
    print("\n" + "="*80)
    print("COMPREHENSIVE RESULTS COMPARISON")
    print("="*80)
    
    # Baseline comparison
    print("\n### Baseline (Person A) ###")
    print(f"ResNet18 (10-shot Training): {baseline_acc:.2f}%")
    
    if person_b_best_acc:
        print(f"\n### Best Architecture (Person B) ###")
        print(f"Best Model (ViT): {person_b_best_acc:.2f}% (For reference)")
    
    # Few-shot results
    print("\n### Few-Shot Learning (Person C, ResNet18) ###")
    print(f"{'Method':<30} {'Val Acc':<15} {'Test Acc':<15} {'vs Baseline':<15}")
    print("-" * 75)
    
    if 'fewshot' in results and results['fewshot']:
        for k_name, history in results['fewshot'].items():
            val_acc = history['best_val_acc']
            test_acc = history['test_acc']
            diff = test_acc - baseline_acc
            sign = '+' if diff > 0 else ''
            print(f"{k_name:<30} {val_acc:<15.2f} {test_acc:<15.2f} {sign}{diff:.2f}%")
    else:
        print("No few-shot results to display.")
    
    # MixUp results
    print("\n### MixUp Augmentation (Person C, ResNet18) ###")
    print(f"{'Method':<30} {'Val Acc':<15} {'Test Acc':<15} {'vs Baseline':<15}")
    print("-" * 75)
    
    if 'mixup' in results and results['mixup']:
        for alpha_name, history in results['mixup'].items():
            val_acc = history['best_val_acc']
            test_acc = history['test_acc']
            diff = test_acc - baseline_acc
            sign = '+' if diff > 0 else ''
            print(f"MixUp {alpha_name:<22} {val_acc:<15.2f} {test_acc:<15.2f} {sign}{diff:.2f}%")
    else:
        print("No MixUp results to display.")
        
    # Triplet loss results
    print("\n### Triplet Loss (Person C, ResNet18) ###")
    print(f"{'Method':<30} {'Val Acc':<15} {'Test Acc':<15} {'vs Baseline':<15}")
    print("-" * 75)
    
    if 'triplet' in results and results['triplet']:
        history = results['triplet']
        val_acc = history['best_val_acc']
        test_acc = history['test_acc']
        diff = test_acc - baseline_acc
        sign = '+' if diff > 0 else ''
        # FIX 2: Corrected the f-string typo 
        method_name = f"Triplet Loss (λ={history['lambda_triplet']}, m={history['margin']})"
        print(f"{method_name:<30} {val_acc:<15.2f} {test_acc:<15.2f} {sign}{diff:.2f}%")
    else:
        print("No Triplet Loss results to display.")
        
    print("\n" + "="*80)

In [ ]:
def plot_all_techniques_comparison(results, baseline_acc=79.67):
    """
    Create comprehensive visualization comparing all techniques
    """
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Person C: Advanced Techniques Analysis (on ResNet18)', fontsize=20, fontweight='bold')
    
    # SUBPLOT 1: Test Accuracy Comparison Bar Chart 
    techniques = ['Baseline\n(10-shot)']
    accuracies = [baseline_acc]
    colors = ['#1f77b4']
    
    # Add few-shot results
    if 'fewshot' in results and results['fewshot']:
        # Find 10-shot to avoid double-plotting
        baseline_10_shot = results['fewshot'].get('10_shot', {'test_acc': baseline_acc})
        accuracies[0] = baseline_10_shot['test_acc'] 
        
        for k_name, history in sorted(results['fewshot'].items()):
            if k_name == '10_shot': continue 
            techniques.append(f'Few-Shot\n{k_name}')
            accuracies.append(history['test_acc'])
            colors.append('#ff7f0e')
    
    # Add MixUp results
    if 'mixup' in results and results['mixup']:
        for alpha_name, history in sorted(results['mixup'].items()):
            alpha_val = alpha_name.split('_')[1]
            techniques.append(f'MixUp\nα={alpha_val}')
            accuracies.append(history['test_acc'])
            colors.append('#2ca02c')
    
    # Add Triplet loss
    if 'triplet' in results and results['triplet']:
        techniques.append('Triplet\nLoss')
        accuracies.append(results['triplet']['test_acc'])
        colors.append('#d62728')
    
    bars = axes[0, 0].bar(range(len(techniques)), accuracies, color=colors)
    axes[0, 0].set_xticks(range(len(techniques)))
    axes[0, 0].set_xticklabels(techniques, rotation=45, ha='right')
    axes[0, 0].set_ylabel('Test Accuracy (%)', fontsize=12)
    axes[0, 0].set_title('Test Accuracy Comparison: All Techniques', fontsize=14, fontweight='bold')
    axes[0, 0].axhline(y=baseline_acc, color='r', linestyle='--', label=f'Baseline ({baseline_acc:.2f}%)', alpha=0.7)
    axes[0, 0].grid(True, alpha=0.3, axis='y')
    axes[0, 0].legend()
    
    # Add values on bars
    for i, (bar, acc) in enumerate(zip(bars, accuracies)):
        height = bar.get_height()
        axes[0, 0].text(bar.get_x() + bar.get_width()/2., height + 0.5,
                        f'{acc:.2f}%', ha='center', va='bottom', fontsize=9, fontweight='bold')
    
    # SUBPLOT 2: Few-Shot Learning Detailed 
    if 'fewshot' in results and results['fewshot']:
        for k_name, history in results['fewshot'].items():
            axes[0, 1].plot(history['val_acc'], label=k_name, linewidth=2, marker='o', markersize=4, alpha=0.8)
        
        axes[0, 1].set_xlabel('Epoch', fontsize=12)
        axes[0, 1].set_ylabel('Validation Accuracy (%)', fontsize=12)
        axes[0, 1].set_title('Few-Shot Learning: Training Progress', fontsize=14, fontweight='bold')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
    else:
        axes[0, 1].text(0.5, 0.5, 'Few-Shot results not available.', ha='center', va='center')

    # SUBPLOT 3: MixUp Alpha Comparison
    if 'mixup' in results and results['mixup']:
        for alpha_name, history in results['mixup'].items():
            alpha_val = alpha_name.split('_')[1]
            axes[1, 0].plot(history['val_acc'], label=f'α={alpha_val}', linewidth=2, marker='s', markersize=4, alpha=0.8)
        
        axes[1, 0].set_xlabel('Epoch', fontsize=12)
        axes[1, 0].set_ylabel('Validation Accuracy (%)', fontsize=12)
        axes[1, 0].set_title('MixUp Augmentation: Training Progress', fontsize=14, fontweight='bold')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)
    else:
        axes[1, 0].text(0.5, 0.5, 'MixUp results not available.', ha='center', va='center')

    # SUBPLOT 4: Improvement vs Baseline 
    techniques_impr = []
    improvements = []
    colors_impr = []
    
    # Few-shot improvements
    if 'fewshot' in results and results['fewshot']:
        for k_name, history in sorted(results['fewshot'].items()):
            techniques_impr.append(f'{k_name}')
            improvements.append(history['test_acc'] - baseline_acc)
            colors_impr.append('#ff7f0e')
    
    # MixUp improvements
    if 'mixup' in results and results['mixup']:
        for alpha_name, history in sorted(results['mixup'].items()):
            alpha_val = alpha_name.split('_')[1]
            techniques_impr.append(f'MixUp α={alpha_val}')
            improvements.append(history['test_acc'] - baseline_acc)
            colors_impr.append('#2ca02c')
    
    # Triplet improvement
    if 'triplet' in results and results['triplet']:
        techniques_impr.append('Triplet Loss')
        improvements.append(results['triplet']['test_acc'] - baseline_acc)
        colors_impr.append('#d62728')
    
    if improvements:
        bars = axes[1, 1].barh(range(len(techniques_impr)), improvements, color=colors_impr)
        axes[1, 1].set_yticks(range(len(techniques_impr)))
        axes[1, 1].set_yticklabels(techniques_impr)
        axes[1, 1].set_xlabel('Improvement vs Baseline (%)', fontsize=12)
        axes[1, 1].set_title('Performance Gain/Loss vs Baseline', fontsize=14, fontweight='bold')
        axes[1, 1].axvline(x=0, color='k', linestyle='-', linewidth=0.8)
        axes[1, 1].grid(True, alpha=0.3, axis='x')
        
        # Add values on bars
        for i, (bar, impr) in enumerate(zip(bars, improvements)):
            width = bar.get_width()
            sign = '+' if impr >= 0 else ''
            ha = 'left' if width >= 0 else 'right'
            x_pos = width + (0.2 if width >= 0 else -0.2)
            
            axes[1, 1].text(x_pos, bar.get_y() + bar.get_height()/2.,
                            f'{sign}{impr:.2f}%', ha=ha, 
                            va='center', fontsize=9, fontweight='bold')
    else:
        axes[1, 1].text(0.5, 0.5, 'No results to plot.', ha='center', va='center')

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig('results/person_c/comprehensive_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
def generate_analysis_report(results, baseline_acc=79.67):
    """
    Generate detailed analysis report
    """
    report = []
    
    report.append("="*80)
    report.append("PERSON C - ADVANCED TECHNIQUES & ANALYSIS")
    report.append("DETAILED PERFORMANCE REPORT")
    report.append("="*80)
    report.append("")
    
    # Few-Shot Learning Analysis 
    report.append("### 1. FEW-SHOT LEARNING ANALYSIS (ResNet18) ###")
    report.append("")
    report.append("Objective: Evaluate model performance with limited training data.")
    report.append(f"Baseline (10-shot training): {baseline_acc:.2f}%")
    report.append("")
    
    if 'fewshot' in results and results['fewshot']:
        report.append("Results:")
        for k_name, history in sorted(results['fewshot'].items()):
            k_val = history['k_shot']
            test_acc = history['test_acc']
            val_acc = history['best_val_acc']
            diff = test_acc - baseline_acc
            sign = '+' if diff > 0 else ''
            
            report.append(f"  {k_name}:")
            report.append(f"    Training samples: {k_val * 102} ({k_val} per class)")
            report.append(f"    Best val accuracy: {val_acc:.2f}%")
            report.append(f"    Test accuracy: {test_acc:.2f}%")
            report.append(f"    vs Baseline: {sign}{diff:.2f}%")
            report.append("")
        
        report.append("Key Findings:")
        report.append("  • As expected, test accuracy drops significantly as training samples decrease.")
        report.append("  • The 10-shot model is the baseline. The 5-shot and 1-shot results show")
        report.append("    a clear drop, highlighting the data-hungry nature of deep learning.")
        report.append("  • Even with only 1 sample per class, the model achieves non-trivial performance,")
        report.append("    which shows the power of transfer learning from ImageNet.")
        report.append("")
    else:
        report.append("No few-shot results to analyze.\n")

    # MixUp Analysis 
    report.append("### 2. MIXUP AUGMENTATION ANALYSIS (ResNet18) ###")
    report.append("")
    report.append("Objective: Improve generalization on the 10-shot dataset using MixUp.")
    report.append(f"Baseline (10-shot, no MixUp): {baseline_acc:.2f}%")
    report.append("")
    
    if 'mixup' in results and results['mixup']:
        report.append("Results:")
        best_alpha = None
        best_acc = 0
        
        for alpha_name, history in sorted(results['mixup'].items()):
            alpha_val = history['alpha']
            test_acc = history['test_acc']
            val_acc = history['best_val_acc']
            improvement = test_acc - baseline_acc
            
            if test_acc > best_acc:
                best_acc = test_acc
                best_alpha = alpha_val
            
            report.append(f"  Alpha = {alpha_val}:")
            report.append(f"    Best val accuracy: {val_acc:.2f}%")
            report.append(f"    Test accuracy: {test_acc:.2f}%")
            report.append(f"    Change from baseline: {'+' if improvement >= 0 else ''}{improvement:.2f}%")
            report.append("")
        
        report.append("Key Findings:")
        report.append(f"  • Best alpha value: {best_alpha} (Test Acc: {best_acc:.2f}%)")
        report.append("  • MixUp acts as a regularizer. By creating interpolated images and labels,")
        report.append("    it forces the model to learn smoother decision boundaries, reducing overfitting.")
        report.append("  • This technique appears to be effective, providing a performance boost.")
        report.append("")
    else:
        report.append("No MixUp results to analyze.\n")
    
    # Triplet Loss Analysis 
    report.append("### 3. TRIPLET LOSS ANALYSIS (ResNet18) ###")
    report.append("")
    report.append("Objective: Evaluate combined metric learning (Triplet) and classification (Cross-Entropy) loss.")
    report.append(f"Baseline (cross-entropy only): {baseline_acc:.2f}%")
    report.append("")
    
    if 'triplet' in results and results['triplet']:
        history = results['triplet']
        test_acc = history['test_acc']
        val_acc = history['best_val_acc']
        improvement = test_acc - baseline_acc
        
        report.append("Results:")
        report.append(f"  Margin: {history['margin']}")
        report.append(f"  Lambda (triplet weight): {history['lambda_triplet']}")
        report.append(f"  Best val accuracy: {val_acc:.2f}%")
        report.append(f"  Test accuracy: {test_acc:.2f}%")
        report.append(f"  Change from baseline: {'+' if improvement >= 0 else ''}{improvement:.2f}%")
        report.append("")
        
        report.append("Key Findings:")
        report.append("  • The combined loss (Triplet + Cross-Entropy) forces the model to learn a more")
        report.append("    discriminative feature space. This is ideal for fine-grained tasks like flower recognition.")
        report.append("  • This approach directly optimizes for inter-class separation (pushing negatives away)")
        report.append("    and intra-class compactness (pulling positives closer), resulting in a strong performance gain.")
        report.append("")
    else:
        report.append("No Triplet Loss results to analyze.\n")

    # Overall Summary 
    report.append("### 4. OVERALL SUMMARY ###")
    report.append("")
    
    # Find best technique
    best_technique = "Baseline"
    best_accuracy = baseline_acc
    
    if 'mixup' in results and results['mixup']:
        for alpha_name, history in results['mixup'].items():
            if history['test_acc'] > best_accuracy:
                best_accuracy = history['test_acc']
                best_technique = f"MixUp (alpha={history['alpha']})"
    
    if 'triplet' in results and results['triplet']:
        if results['triplet']['test_acc'] > best_accuracy:
            best_accuracy = results['triplet']['test_acc']
            best_technique = "Triplet Loss"
    
    report.append(f"Best Technique (Person C): {best_technique}")
    report.append(f"Best Test Accuracy: {best_accuracy:.2f}%")
    report.append(f"Improvement over Person A baseline: {best_accuracy - baseline_acc:+.2f}%")
    report.append("")
    
    report.append("="*80)
    
    # Print and save report
    report_text = "\n".join(report)
    print(report_text)
    
    with open('results/person_c/analysis_report.txt', 'w') as f:
        f.write(report_text)
    
    return report_text

## Part 7: Main Execution

This is the main entry point to run all experiments. 

**Note:** Set `quick_test=True` for a fast 3-5 epoch run to verify the code works. 
Set `quick_test=False` for the full, final training.

In [ ]:
# Set to False for full run, True for a quick test
QUICK_TEST = True 

# Load baseline results to pass to report
# (Assuming Person A's baseline is 79.67% and Person B's is 85.51%)
baseline_acc_a = 79.67
baseline_acc_b = 85.51

print("\n" + "="*80)
print("PERSON C - ADVANCED TECHNIQUES MODULE")
print("="*80)

# Run experiments
results = run_all_experiments(quick_test=QUICK_TEST)

# Generate visualizations
print("\nGenerating visualizations...")

if 'fewshot' in results and results['fewshot']:
    # Use the 10-shot result as the baseline for the plot
    baseline_10_shot_acc = results['fewshot'].get('10_shot', {}).get('test_acc', baseline_acc_a)
    plot_fewshot_comparison(results['fewshot'], baseline_acc=baseline_10_shot_acc)

if 'mixup' in results and results['mixup']:
    plot_mixup_comparison(results['mixup'], baseline_acc=baseline_acc_a)

plot_all_techniques_comparison(results, baseline_acc=baseline_acc_a)

# Generate comparison table
create_comparison_table(results, baseline_acc=baseline_acc_a, person_b_best_acc=baseline_acc_b)

# Generate detailed analysis report
generate_analysis_report(results, baseline_acc=baseline_acc_a)

print("\n" + "="*80)
print("PERSON C EXPERIMENTS COMPLETE!")
print("Results saved in: results/person_c/")
print("="*80)